# Notebook 1: Linear decoders

In [ ]:
!pip install scipy==1.7.3
!pip install mne==0.24.1

!git clone https://github.com/Mike-boop/mldecoders.git

import os
os.chdir('mldecoders')

!python setup.py install

In [ ]:
# Download the data from the CNSP web server 
# This may take several minutes

!curl https://www.data.cnspworkshop.net/data/thornton_data/data.h5 --output data.h5
data_dir = 'data.h5'

In [ ]:
from IPython.display import Audio, display, Image
from scipy.signal import hilbert
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np
from mne.filter import filter_data
import h5py
from pipeline.ridge import Ridge
import pickle

np.random.seed(0)

# Intro

In this tutorial we will be using EEG data that was recorded whilst a participant attended to continuous speech (in the form of audiobooks). Below is an example of a typical experimental setup used to acquire this sort of data, as well as an example audiobook from our dataset.

<img src="../images/cnsp_experiment.png" width="400"/>

In [ ]:
sound_file = 'cnsp_workshop_tutorial/data/AUNP01.wav'
srate, speech = wavfile.read(sound_file)
wn = Audio(sound_file, autoplay=True, rate=srate)
display(wn)

We can study what features of the stimulus are encoded in the EEG recordings by attempting to reconstruct (or decode) them:

<img src="../images/cnsp_bw_model.png" width="800"/>

The speech envelope (essentially the amplitude) turns out to be rather strongly encoded in EEG recordings. You can see how the speech envelope relates to the original waveform below:

In [ ]:
def get_envelope(x, fs):
    envelope = np.abs(hilbert(x))
    return filter_data(envelope, fs, None, 50)

t = np.arange(len(speech))/srate
plt.plot(t, speech, label = 'speech')
plt.plot(t, get_envelope(speech, srate), label = 'speech envelope')
plt.xlim(0, 10)

plt.xlabel('Time [s]')
plt.ylabel('Amplitude [au]')
plt.title('Plot of stimulus and stimulus envelope')

In the linear decoding approach, we take a linear combination (or weighted sum) of the EEG recordings within a certain time window in order to predict the speech envelope at the start of the window:

<img src="../images/cnsp_maths.png" width="800"/>

The traditional method for finding the optimal weights/parameters is ridge regression. This is implemented in popular toolboxes such as the mTRF toolbox and mne-Python. We also provide an implementation which will be explored in this notebook. The famous formula for the ridge parameters is as follows:

<img src="../images/cnsp_ridge.png" width="400"/>

# Data, availability, and preprocessing

In the tutorial we will use EEG data recorded from 13 participants. Each participant listened to a single speaker narrate audiobook chapters in English. There were 15 trials per participant. Full details of the data acquisition procedure are given in [Hugo's paper](https://direct.mit.edu/jocn/article-abstract/32/1/155/95401/Cortical-Tracking-of-Surprisal-during-Continuous).

A version of this dataset is freely available on [figshare](https://figshare.com/projects/Cortical_Tracking_of_Surprisal_during_Continuous_Speech_Comprehension/66596). If you are interested in using the unprocessed dataset for your research, feel free to contact me at mdt20@ic.ac.uk. We plan to release the unprocessed recordings, as well as another dataset with more interesting stimuli, in the near future.

The EEG data used in this tutorial has already been preprocessed. You can find details of the preprocessing procedure in our paper, as well as in the `data_preprocessing` directory in this repository. The sampling rate of the preprocessed data is 125 Hz, and the EEG data have been filtered between 0.5 Hz and 8 Hz. We have used broadband stimulus envelopes, lowpass filtered below 50 Hz.

The frequency ranges of the data are verified for one of the participants below:

In [ ]:
with h5py.File(data_dir, 'r') as f:
    eeg = f['eeg/P00/part1'][:]
    envelope = f['stim/part1'][:]

In [ ]:
fig, axs = plt.subplots(1,2)

for i in range(63): 
    axs[0].psd(eeg[i], Fs=125);
    
axs[1].psd(envelope, Fs=125);

# Linear models

We would like to relate the EEG recordings to the speech envelopes. One way of doing this is via backward modelling, otherwise known as stimulus-reconstruction. We will begin by using an approach which is similar to the TRF model, but predicts the speech envelope given a temporal context of EEG recordings using a linear filter.

We will fit the linear filter via ridge regression, which is common practice in the field. In the paper, we used the first nine story parts as training data, the next 3 as validation data for tuning the regularisation parameter, and the final 3 as testing data. We could also have chosen to perform cross-validation here.

## Fitting a linear model for a single participant

First, we go over fitting a linear model in detail, for a single participant. We choose the first participant (0) by default.

In the paper, we used nine EEG trials to train the linear models (via ridge regression). Here, to speed things up, we will use three trials to fit the model parameters. As in the paper, three trials will be used for 'validation' - that is, for choosing the regularisation hyperparameter $\lambda$. The remaining three trials will be used to evaulate the fitted with the chosen hyperparameter.

In [ ]:
participant = 0

train_parts = range(3)#range(9)
val_parts = range(9,12)
test_parts = range(12, 15)

# try 15 regularisation parameters spaced evenly on a log scale between 1e-7 and 1e7
regularisation_parameters = np.logspace(-7, 7, 15)

Concatenating the trials in the training, validation, and testing datasets, respectively:

In [ ]:
with h5py.File(data_dir, 'r') as f:
    X_train = np.hstack([f[f'eeg/P0{participant}/part{j}'][:] for j in train_parts])
    y_train = np.hstack([f[f'stim/part{j}'][:] for j in train_parts])
    
    X_val = np.hstack([f[f'eeg/P0{participant}/part{j}'][:] for j in val_parts])
    y_val = np.hstack([f[f'stim/part{j}'][:] for j in val_parts])
    
    X_test = np.hstack([f[f'eeg/P0{participant}/part{j}'][:] for j in test_parts])
    y_test = np.hstack([f[f'stim/part{j}'][:] for j in test_parts])

To fit the model, we have to state the number of lags used in the spatiotemporal window, which is 50 in this case. Note that since the sampling rate is 125 Hz, this corresponds to a window duration of about 0.4 s.

We can also set an offset between the start of the window and the predicted envelope value, which is defined by `start_lag`. This is set to zero here, since we assume causality. This parameter is useful for fitting encoding models.

In [ ]:
# instantiate the model
mdl = Ridge(start_lag=0, end_lag=50, alpha=regularisation_parameters)

# fit the model parameters via ridge regression, for all regularisation parameters
mdl.fit(X_train.T, y_train[:, np.newaxis])

# evaluate the model on the validation dataset for every regularisation parameter, and select the best value
val_scores = mdl.model_selection(X_val.T, y_val[:, np.newaxis])

# test the model on the test dataset
test_score = mdl.score(X_test.T, y_test[:, np.newaxis])[0]

In [ ]:
plt.plot(regularisation_parameters, val_scores)
plt.xscale('log')

plt.title('Reconstruction accuracies for validation dataset')
plt.ylabel('Validation score (Pearson r)')
plt.xlabel('Regularisation parameter')

print('Best regularisation parameter: ', mdl.alphas[mdl.best_alpha_idx])
print('Test score (correlation):', test_score)

## Fitting linear models for several participants

Now we're ready to fit linear decoders for all of the participants! In the following cell, you can also see how the trained linear models and their predictions are saved to disk for use later. We also obtain 'null scores', which are explained next.

To speed things up, we only fit linear models for the first 5 participants. You are welcome to experiment with all 13 in your free time!

In [ ]:
# This cell should take around 6 minutes to run

test_scores = []
null_scores = []

with h5py.File(data_dir, 'r') as f:
    
    mdl = Ridge(start_lag=0, end_lag=50, alpha=regularisation_parameters)

    for participant in range(5):
        
        print('Fitting model for participant:', participant)
        
        # training using all regularisation parameters
        X_train = np.hstack([f[f'eeg/P0{participant}/part{j}'][:] for j in train_parts])
        y_train = np.hstack([f[f'stim/part{j}'][:] for j in train_parts])
        mdl.fit(X_train.T, y_train[:, np.newaxis])
        
        # select best regularisation parameter
        X_val = np.hstack([f[f'eeg/P0{participant}/part{j}'][:] for j in val_parts])
        y_val = np.hstack([f[f'stim/part{j}'][:] for j in val_parts])
        val_scores = mdl.model_selection(X_val.T, y_val[:, np.newaxis])
        
        # save the trained model
        pickle.dump(mdl, open(f"cnsp_workshop_tutorial/results/linear_models/P{participant:02d}_ridge.pk", "wb"))
        
        # get and save predicted speech envelope
        X_test = np.hstack([f[f'eeg/P0{participant}/part{j}'][:] for j in test_parts])
        y_test = np.hstack([f[f'stim/part{j}'][:] for j in test_parts])
        
        test_predictions = mdl.predict(X_test.T)
        np.save(f"cnsp_workshop_tutorial/results/linear_models/P{participant:02d}_predictions.npy", test_predictions[0])
        
        # compute the correlation between the predictions and the true speech envelope
        test_score = mdl.score(X_test.T, y_test[:, np.newaxis])[0]
        test_scores.append(test_score)
        
        # compute the null correlations
        null_score = mdl.score(X_test.T, y_test[::-1, np.newaxis])[0]
        null_scores.append(null_score)

## Results

In the last two lines of the above cell, we computed 'null scores' by time-reversing the true speech envelope and correlating it against the predicted speech envelope. Since there should be no systematic correlation between the predicted speech envelope and the time-reversed speech envelope, the resulting 'null scores' allow us to estimate the noise level of the reconstruction scores (Pearson correlation coefficients). This allows us to assess whether the (non-null) reconstruction scores are due to chance correlations, or whether the speech envelope really can be decoded from the EEG recordings.

The reconstruction scores and null scores for each participant are plotted below:

In [ ]:
width = .2
x = np.arange(5)
plt.bar(x-width/2, test_scores, width=width, label='correlations')
plt.bar(x+width/2, null_scores, width=width, label='null correlations')

plt.title("Correlation coefficients between the predicted and actual speech envelopes")
plt.ylabel('Reconstruction score (Pearson)')
plt.xlabel("Participant")
plt.legend()

We should check that the reconstruction scores are significantly different to the null distribution. We can do this with a t-test. The p-value turns out to be quite significant:

In [ ]:
from scipy.stats import ttest_ind
print(ttest_ind(test_scores, null_scores, alternative='greater'))

# Exercises

## 1. Find regularisation curves for several of the participants.

The regularisation curve is a plot of validation score (Pearson's correlation coefficient) against regularisation parameter, as shown earlier in this notebook. You should be able to modify the code provided, in order to save the `val_scores` array for each of the participants. Plot these curves on the same axes.

 __To speed things up, use just the first three trials to train the linear models. Plot the regularisation curves for no more than five participants.__

## 2. Questions about the regularisation curves:

- Is the optimal regularisation parameter the same for all participants?
- What is the mean optimal regularisation parameter?
- What do you notice about the regularisation curves for small and large values of the regularisaiton parameter? Can you explain this?

## 3. (Extension) Try fitting forward models using the Ridge class. You'll need to change the `start_lag` parameter.